In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Colors_3.csv')
df.groupby('Cat_Name').count()['Cat_Num']

Cat_Name
(Other)                          249
Aircraft                         309
Animal, Accessory                127
Animal, Air                       94
Animal, Body Part                702
Animal, Dinosaur                  55
Animal, Land                     438
Animal, Water                     97
Antenna                           77
Arm                               77
Bar                              355
Baseplate                        172
Baseplate, Raised                 78
Baseplate, Road                  107
Belville                         199
Belville, Figure Accessory       109
Bionicle                        1682
Bionicle, Kanohi Mask            369
Boat                             385
Bracket                          178
Brick                           1185
Brick, Arch                      411
Brick, Decorated                2161
Brick, Modified                  982
Brick, Modified, Decorated       306
Brick, Promotional               226
Brick, Round                 

In [3]:
df.shape

(27115, 8)

In [4]:
df.tail()

,Part_Num,Col_Num,Cat_Num,Cat_Name,Part_Name,Col_Name,Num_Sets,Num_Sold
27110,bb748,103,20.0,"Minifig, Body Part","Arm, Short, Curved with Hand",Bright Light Yellow,4.0,0.0
27111,bb748,154,20.0,"Minifig, Body Part","Arm, Short, Curved with Hand",Lavender,6.0,0.0
27112,bb748,152,20.0,"Minifig, Body Part","Arm, Short, Curved with Hand",Light Aqua,2.0,0.0
27113,bb748,150,20.0,"Minifig, Body Part","Arm, Short, Curved with Hand",Medium Dark Flesh,2.0,0.0
27114,bb748,157,20.0,"Minifig, Body Part","Arm, Short, Curved with Hand",Medium Lavender,2.0,0.0


In [5]:
col = 'Num_Sold'
df.groupby('Col_Name').agg({col:sum}).sort_values(col, ascending=False).head(10)

,Num_Sold
Col_Name,
Black,488740.0
Light Bluish Gray,393396.0
White,343948.0
Dark Bluish Gray,292122.0
Red,180192.0
Tan,143434.0
Reddish Brown,132665.0
Yellow,124630.0
Blue,87814.0


In [6]:
col = 'Num_Sets'
df.groupby('Col_Name').agg({col:sum}).sort_values(col, ascending=False).head(10)

,Num_Sets
Col_Name,
Black,35436.0
White,30015.0
Red,23213.0
Yellow,18892.0
Light Bluish Gray,16435.0
Dark Bluish Gray,13506.0
Blue,13147.0
Light Gray,7023.0
Green,6287.0


In [7]:
col = 'Num_Sold'
df.groupby('Cat_Name').agg({col:sum}).sort_values(col, ascending=False).head(10)

,Num_Sold
Cat_Name,
Brick,710489.0
"Brick, Modified",401528.0
"Brick, Round",171423.0
Bar,151243.0
Bracket,105958.0
"Brick, Arch",79622.0
Cone,77046.0
Dish,65834.0
Bionicle,64444.0


In [8]:
col = 'Num_Sold'
df.groupby('Cat_Name').agg({col:'median'}).sort_values(col, ascending=False).head(10)

,Num_Sold
Cat_Name,
Lever,914.0
"Plate, Round",192.0
Tile,150.5
"Wedge, Plate",141.5
Plate,141.0
"Minifig, Weapon",137.0
Plant,123.5
"Slope, Inverted",107.0
Fence,105.5


In [9]:
import re
import requests
from bs4 import BeautifulSoup


total = re.compile(':.(\d+)')
qty = re.compile('Total Qty:(\d+)')
min_price = re.compile('Min Price:EUR.(\d+\.*\d+)')
avg_price = re.compile('Avg Price:EUR.(\d+\.*\d+)')
qty_price = re.compile('Qty Avg Price:EUR.(\d+\.*\d+)')
max_price = re.compile('Max Price:EUR.(\d+\.*\d+)')

In [10]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
url = 'https://www.bricklink.com/catalogPG.asp?P=2335pb110&ColorID=5'

In [11]:
bs = BeautifulSoup(requests.get(url, headers=headers).text, 'lxml')

In [12]:
price_dict = {'Sale_New':{'t':0, 'q':0, 'mi_p':0, 'a_p':0, 'q_p':0, 'ma_p':0},
              'Sale_Used':{'t':0, 'q':0, 'mi_p':0, 'a_p':0, 'q_p':0, 'ma_p':0},
              'Store_New':{'t':0, 'q':0, 'mi_p':0, 'a_p':0, 'q_p':0, 'ma_p':0},
              'Store_Used':{'t':0, 'q':0, 'mi_p':0, 'a_p':0, 'q_p':0, 'ma_p':0}}

In [13]:
texts = [elem.get_text().replace('-', '0') for elem in bs.find('tr', {'bgcolor':'#C0C0C0'}).findAll('td', {'valign':'TOP'})]

In [14]:
texts

['(Unavailable)',
 'Times Sold:\xa08Total Qty:8Min Price:EUR\xa00.40Avg Price:EUR\xa00.70Qty Avg Price:EUR\xa00.70Max Price:EUR\xa01.16',
 'Total Lots:\xa02Total Qty:13Min Price:EUR\xa00.45Avg Price:EUR\xa03.38Qty Avg Price:EUR\xa00.90Max Price:EUR\xa06.31',
 'Total Lots:\xa034Total Qty:44Min Price:EUR\xa00.33Avg Price:EUR\xa00.81Qty Avg Price:EUR\xa00.96Max Price:EUR\xa04.09']

In [15]:
tmp = [True if 'Sold' in text else False for text in texts]

if len(tmp)==4:
    order = ['Sale_New', 'Sale_Used', 'Store_New', 'Store_Used']
elif tmp[0]==True and tmp[1]==True and len(tmp)==3:
    order = ['Sale_New', 'Sale_Used', 'Store_Used']
elif tmp[0]==True and tmp[1]==False and len(tmp)==3:
    order = ['Sale_Used', 'Store_New', 'Store_Used']
elif tmp[0]==False and tmp[1]==False and len(tmp)==2:
    order = ['Store_New', 'Store_Used']
elif tmp[0]==True and tmp[1]==True and len(tmp)==2:
    order = ['Sale_New', 'Sale_Used']
else:
    order = []

In [16]:
for o, t in zip(order, texts):
    try:
        price_dict[o]['t'] = re.search(total, text)[1]
        price_dict[o]['q'] = re.search(qty, text)[1]
        price_dict[o]['mi_p'] = re.search(min_price, text)[1]
        price_dict[o]['a_p'] = re.search(avg_price, text)[1]
        price_dict[o]['q_p'] = re.search(qty_price, text)[1]
        price_dict[o]['ma_p'] = re.search(max_price, text)[1]
    except:
        pass

In [17]:
col, val = zip(*[(i+'_'+x, y) for i, items in price_dict.items() for x, y in items.items()])

In [18]:
import pandas as pd